In [1]:
import pickle
from pathlib import Path
from pprint import pprint
import numpy as np
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.utils import common_utils
import torch

tv = None
try:
  import cumm.tensorview as tv
except:
  pass

from nw_models.voxelnext import VoxelNext
from functools import partial

In [2]:
root_path = Path("/home/cuda_pp/OpenPCDet/data/nuscenes/v1.0-mini")

info_path = Path("/home/cuda_pp/OpenPCDet/data/nuscenes/v1.0-mini/nuscenes_infos_10sweeps_val.pkl")

with open(info_path, 'rb') as f:
  infos = pickle.load(f)
  info = infos[10]
  # pprint(infos[1])

In [3]:
def remove_ego_points(points, center_radius=1.0):
  mask = ~((np.abs(points[:, 0]) < center_radius) & (np.abs(points[:, 1]) < center_radius))
  return points[mask]

def get_sweep(sweep_info):
    lidar_path = root_path / sweep_info['lidar_path']
    points_sweep = np.fromfile(str(lidar_path), dtype=np.float32, count=-1).reshape([-1, 5])[:, :4]
    points_sweep = remove_ego_points(points_sweep).T
    if sweep_info['transform_matrix'] is not None:
        num_points = points_sweep.shape[1]
        points_sweep[:3, :] = sweep_info['transform_matrix'].dot(
            np.vstack((points_sweep[:3, :], np.ones(num_points))))[:3, :]

    cur_times = sweep_info['time_lag'] * np.ones((1, points_sweep.shape[1]))
    return points_sweep.T, cur_times.T

def get_lidar_with_sweeps(info, max_sweeps=1):
    lidar_path = root_path / info['lidar_path']
    points = np.fromfile(str(lidar_path), dtype=np.float32, count=-1).reshape([-1, 5])[:, :4]

    sweep_points_list = [points]
    sweep_times_list = [np.zeros((points.shape[0], 1))]

    for k in np.random.choice(len(info['sweeps']), max_sweeps - 1, replace=False):
        points_sweep, times_sweep = get_sweep(info['sweeps'][k])
        sweep_points_list.append(points_sweep)
        sweep_times_list.append(times_sweep)

    points = np.concatenate(sweep_points_list, axis=0)
    times = np.concatenate(sweep_times_list, axis=0).astype(points.dtype)

    points = np.concatenate((points, times), axis=1)
    return points

In [4]:

cfg_file = "/home/cuda_pp/OpenPCDet/tools/cfgs/nuscenes_models/cbgs_voxel0075_voxelnext.yaml"

cfg_from_yaml_file(cfg_file, cfg)
cfg.TAG = Path(cfg_file).stem
cfg.EXP_GROUP_PATH = '/'.join(cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

dataset_cfg = cfg.DATA_CONFIG

print("############# DATASET CONFIG ###################\n", dataset_cfg)

############# DATASET CONFIG ###################
 {'DATASET': 'NuScenesDataset', 'DATA_PATH': '../data/nuscenes', 'VERSION': 'v1.0-mini', 'MAX_SWEEPS': 10, 'PRED_VELOCITY': True, 'SET_NAN_VELOCITY_TO_ZEROS': True, 'FILTER_MIN_POINTS_IN_GT': 1, 'DATA_SPLIT': {'train': 'train', 'test': 'val'}, 'INFO_PATH': {'train': ['nuscenes_infos_10sweeps_train.pkl'], 'test': ['nuscenes_infos_10sweeps_val.pkl']}, 'POINT_CLOUD_RANGE': [-54.0, -54.0, -5.0, 54.0, 54.0, 3.0], 'BALANCED_RESAMPLING': True, 'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'], 'AUG_CONFIG_LIST': [{'NAME': 'gt_sampling', 'DB_INFO_PATH': ['nuscenes_dbinfos_10sweeps_withvelo.pkl'], 'USE_SHARED_MEMORY': False, 'DB_DATA_PATH': ['nuscenes_dbinfos_10sweeps_withvelo_global.pkl.npy'], 'PREPARE': {'filter_by_min_points': ['car:5', 'truck:5', 'construction_vehicle:5', 'bus:5', 'trailer:5', 'barrier:5', 'motorcycle:5', 'bicycle:5', 'pedestrian:5', 'traffic_cone:5']}, 'SAMPLE_GROUPS': ['car:2', 'truck:2', 'construction_vehicle:2', 'bus

In [5]:
points = get_lidar_with_sweeps(info, max_sweeps=dataset_cfg.MAX_SWEEPS)

In [6]:
# shuffle point indexes
# shuffle_idx = np.random.permutation(points.shape[0])
# points = points[shuffle_idx]

In [7]:
class VoxelGeneratorWrapper():
  def __init__(
          self, vsize_xyz, coors_range_xyz, num_point_features, max_num_points_per_voxel,
          max_num_voxels):
    try:
      from spconv.utils import VoxelGeneratorV2 as VoxelGenerator
      self.spconv_ver = 1
    except:
      try:
        from spconv.utils import VoxelGenerator
        self.spconv_ver = 1
      except:
        from spconv.utils import Point2VoxelCPU3d as VoxelGenerator
        self.spconv_ver = 2

    if self.spconv_ver == 1:
      self._voxel_generator = VoxelGenerator(
          voxel_size=vsize_xyz,
          point_cloud_range=coors_range_xyz,
          max_num_points=max_num_points_per_voxel,
          max_voxels=max_num_voxels
      )
    else:
      self._voxel_generator = VoxelGenerator(
          vsize_xyz=vsize_xyz,
          coors_range_xyz=coors_range_xyz,
          num_point_features=num_point_features,
          max_num_points_per_voxel=max_num_points_per_voxel,
          max_num_voxels=max_num_voxels
      )

  def generate(self, points):
    if self.spconv_ver == 1:
      voxel_output = self._voxel_generator.generate(points)
      if isinstance(voxel_output, dict):
        voxels, coordinates, num_points = \
            voxel_output['voxels'], voxel_output['coordinates'], voxel_output['num_points_per_voxel']
      else:
        voxels, coordinates, num_points = voxel_output
    else:
      assert tv is not None, f"Unexpected error, library: 'cumm' wasn't imported properly."
      voxel_output = self._voxel_generator.point_to_voxel(tv.from_numpy(points))
      tv_voxels, tv_coordinates, tv_num_points = voxel_output
      # make copy with numpy(), since numpy_view() will disappear as soon as the generator is deleted
      voxels = tv_voxels.numpy()
      coordinates = tv_coordinates.numpy()
      num_points = tv_num_points.numpy()
    return voxels, coordinates, num_points

In [8]:
class PointFeatureEncoder(object):
    def __init__(self, config, point_cloud_range=None):
        super().__init__()
        self.point_encoding_config = config
        assert list(self.point_encoding_config.src_feature_list[0:3]) == ['x', 'y', 'z']
        self.used_feature_list = self.point_encoding_config.used_feature_list
        self.src_feature_list = self.point_encoding_config.src_feature_list
        self.point_cloud_range = point_cloud_range

    @property
    def num_point_features(self):
        return getattr(self, self.point_encoding_config.encoding_type)(points=None)

    def forward(self, points):
        """
        Args:
            data_dict:
                points: (N, 3 + C_in)
                ...
        Returns:
            data_dict:
                points: (N, 3 + C_out),
                use_lead_xyz: whether to use xyz as point-wise features
                ...
        """
        points, use_lead_xyz = getattr(self, self.point_encoding_config.encoding_type)(
            points
        )
       
        if self.point_encoding_config.get('filter_sweeps', False) and 'timestamp' in self.src_feature_list:
            max_sweeps = self.point_encoding_config.max_sweeps
            idx = self.src_feature_list.index('timestamp')
            dt = np.round(points[:, idx], 2)
            max_dt = sorted(np.unique(dt))[min(len(np.unique(dt))-1, max_sweeps-1)]
            points = points[dt <= max_dt]
        
        return points, use_lead_xyz

    def absolute_coordinates_encoding(self, points=None):
        if points is None:
            num_output_features = len(self.used_feature_list)
            return num_output_features

        assert points.shape[-1] == len(self.src_feature_list)
        point_feature_list = [points[:, 0:3]]
        for x in self.used_feature_list:
            if x in ['x', 'y', 'z']:
                continue
            idx = self.src_feature_list.index(x)
            point_feature_list.append(points[:, idx:idx+1])
        point_features = np.concatenate(point_feature_list, axis=1)
        
        return point_features, True

In [9]:
point_feature_encoder = PointFeatureEncoder(
    dataset_cfg.POINT_FEATURE_ENCODING,
    point_cloud_range=dataset_cfg.POINT_CLOUD_RANGE
)
num_point_features = point_feature_encoder.num_point_features
print("points shape: ", points.shape)
points, use_lead_xyz = point_feature_encoder.forward(points)
print("points shape after encoding: ", points.shape)

points shape:  (269138, 5)
points shape after encoding:  (269138, 5)


In [10]:
class DataProcessor:
  def __init__(self, processor_configs, point_cloud_range, training, num_point_features):
    self.point_cloud_range = point_cloud_range
    self.training = training
    self.num_point_features = num_point_features
    self.mode = 'train' if training else 'test'
    self.grid_size = self.voxel_size = None
    self.data_processor_queue = []

    self.voxel_generator = None

  def mask_points_and_boxes_outside_range(self, points):    
    mask = common_utils.mask_points_by_range(points, self.point_cloud_range)
    return points[mask]

  def shuffle_points(self, points, config=None):
    if config.SHUFFLE_ENABLED[self.mode]:
        shuffle_idx = np.random.permutation(points.shape[0])
        points = points[shuffle_idx]

    return points

  def double_flip(self, points):
    # y flip
    points_yflip = points.copy()
    points_yflip[:, 1] = -points_yflip[:, 1]

    # x flip
    points_xflip = points.copy()
    points_xflip[:, 0] = -points_xflip[:, 0]

    # x y flip
    points_xyflip = points.copy()
    points_xyflip[:, 0] = -points_xyflip[:, 0]
    points_xyflip[:, 1] = -points_xyflip[:, 1]

    return points_yflip, points_xflip, points_xyflip

  def transform_points_to_voxels(self, points, use_lead_xyz=False, config=None):
    if self.voxel_size is None:
       self.voxel_size = config.VOXEL_SIZE

    if self.grid_size is None:
        grid_size = (self.point_cloud_range[3:6] - self.point_cloud_range[0:3]) / np.array(self.voxel_size)
        self.grid_size = np.round(grid_size).astype(np.int64)

    if self.voxel_generator is None:
        self.voxel_generator = VoxelGeneratorWrapper(
            vsize_xyz=self.voxel_size,
            coors_range_xyz=self.point_cloud_range,
            num_point_features=self.num_point_features,
            max_num_points_per_voxel=config.MAX_POINTS_PER_VOXEL,
            max_num_voxels=config.MAX_NUMBER_OF_VOXELS[self.mode],
        )

    voxel_output = self.voxel_generator.generate(points)
    voxels, coordinates, num_points = voxel_output

    if not use_lead_xyz:
        voxels = voxels[..., 3:]  # remove xyz in voxels(N, 3)

    if config.get('DOUBLE_FLIP', False):
        voxels_list, voxel_coords_list, voxel_num_points_list = [voxels], [coordinates], [num_points]
        points_yflip, points_xflip, points_xyflip = self.double_flip(points)
        points_list = [points_yflip, points_xflip, points_xyflip]
        keys = ['yflip', 'xflip', 'xyflip']
        for i, key in enumerate(keys):
            voxel_output = self.voxel_generator.generate(points_list[i])
            voxels, coordinates, num_points = voxel_output

            if not use_lead_xyz:
                voxels = voxels[..., 3:]
            voxels_list.append(voxels)
            voxel_coords_list.append(coordinates)
            voxel_num_points_list.append(num_points)

        return voxels_list, voxel_coords_list, voxel_num_points_list
    
    return voxels, coordinates, num_points

  def sample_points(self, points, config=None):
    num_points = config.NUM_POINTS[self.mode]
    if num_points == -1:
        return points

    if num_points < len(points):
        pts_depth = np.linalg.norm(points[:, 0:3], axis=1)
        pts_near_flag = pts_depth < 40.0
        far_idxs_choice = np.where(pts_near_flag == 0)[0]
        near_idxs = np.where(pts_near_flag == 1)[0]
        choice = []
        if num_points > len(far_idxs_choice):
            near_idxs_choice = np.random.choice(near_idxs, num_points - len(far_idxs_choice), replace=False)
            choice = np.concatenate((near_idxs_choice, far_idxs_choice), axis=0) \
                if len(far_idxs_choice) > 0 else near_idxs_choice
        else: 
            choice = np.arange(0, len(points), dtype=np.int32)
            choice = np.random.choice(choice, num_points, replace=False)
        np.random.shuffle(choice)
    else:
        choice = np.arange(0, len(points), dtype=np.int32)
        if num_points > len(points):
            extra_choice = np.random.choice(choice, num_points - len(points), replace=False)
            choice = np.concatenate((choice, extra_choice), axis=0)
        np.random.shuffle(choice)
    return points[choice]


  

In [11]:
point_cloud_range = np.array(dataset_cfg.POINT_CLOUD_RANGE, dtype=np.float32)
data_p = DataProcessor(dataset_cfg.DATA_PROCESSOR, point_cloud_range, False, num_point_features)

In [12]:
for dp_cfg in dataset_cfg.DATA_PROCESSOR:
    if dp_cfg.NAME == 'mask_points_and_boxes_outside_range':
        points = data_p.mask_points_and_boxes_outside_range(points)
    elif dp_cfg.NAME == 'shuffle_points':
        points = data_p.shuffle_points(points, config=dp_cfg)
    elif dp_cfg.NAME == 'transform_points_to_voxels':
        voxels, coordinates, num_points = data_p.transform_points_to_voxels(points, use_lead_xyz, config=dp_cfg)
    else:
        raise ValueError('unimplemented processor function')

In [13]:
# pad coordinates
coordinates = np.pad(coordinates, ((0, 0), (1, 0)), mode='constant', constant_values=0)

voxels = torch.from_numpy(voxels).float().cuda()
coordinates = torch.from_numpy(coordinates).float().cuda()
num_points = torch.from_numpy(num_points).float().cuda()

print('v size: ', voxels.size())
print('v num pts: ', num_points.size())
print('v coords: ', coordinates.size())

v size:  torch.Size([108911, 10, 5])
v num pts:  torch.Size([108911])
v coords:  torch.Size([108911, 3])


In [14]:
# def mask_points_outside_range(points, point_cloud_range):
#   point_cloud_range = np.array(dataset_cfg.POINT_CLOUD_RANGE, dtype=np.float32)
#   mask = common_utils.mask_points_by_range(points, point_cloud_range)
#   return points[mask]

# def shuffle_points(points):
#   shuffle_idx = np.random.permutation(points.shape[0])
#   return points[shuffle_idx]

# points = mask_points_outside_range(points, dataset_cfg.POINT_CLOUD_RANGE)
# points = shuffle_points(points)


In [15]:
# # transform points to voxels
# voxel_gen_config = next(x for x in dataset_cfg.DATA_PROCESSOR if x.NAME == 'transform_points_to_voxels')
# voxel_size = voxel_gen_config.VOXEL_SIZE
# point_cloud_range = np.array(dataset_cfg.POINT_CLOUD_RANGE, dtype=np.float32)
# grid_size = (point_cloud_range[3:6] - point_cloud_range[0:3]) / np.array(voxel_gen_config.VOXEL_SIZE)

# voxel_generator = VoxelGeneratorWrapper(
#     vsize_xyz=voxel_size,
#     coors_range_xyz=point_cloud_range,
#     num_point_features=num_point_features,
#     max_num_points_per_voxel=voxel_gen_config.MAX_POINTS_PER_VOXEL,
#     max_num_voxels=voxel_gen_config.MAX_NUMBER_OF_VOXELS['test'],
# )

# voxels, coordinates, num_points = voxel_generator.generate(points)
# if not use_lead_xyz:
#     voxels = voxels[..., 3:]

# voxels = torch.from_numpy(voxels).float().cuda()
# coordinates = torch.from_numpy(coordinates).float().cuda()
# num_points = torch.from_numpy(num_points).float().cuda()

# print('v size: ', voxels.size())
# print('v num pts: ', num_points.size())
# print('v coords: ', coordinates.size())

In [16]:
model = VoxelNext(cfg.MODEL, cfg.CLASS_NAMES, num_point_features,
                  data_p.grid_size, point_cloud_range, data_p.voxel_size)

In [17]:
model.cuda()
model.eval()

with torch.no_grad():
  pred_dicts, ret_dict = model(1, voxels, num_points, coordinates)

AssertionError: spatial shape must equal to ndim